In [71]:
import requests
import streamlit as st
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import gspread
import pandas as pd
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials

load_dotenv(dotenv_path='config.env')  # 👈 important ici

True

In [72]:
SHOP_NAME = os.getenv("SHOP_NAME_TEC")
ACCESS_TOKEN = os.getenv("SHOPIFY_API_TOKEN")
SHOP_NAME_LOCAL = os.getenv("SHOP_NAME_LOCAL")

HEADERS = {
    "X-Shopify-Access-Token": ACCESS_TOKEN,
    "Content-Type": "application/json"
}

GRAPHQL_URL = f"https://{SHOP_NAME}/admin/api/2024-04/graphql.json"

BASE_URL = f"https://{SHOP_NAME_LOCAL}/admin/api/2024-04"

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("gspread_key.json", scope)
client = gspread.authorize(creds)


In [55]:
# Simple requête test GraphQL
query = """
{
  shop {
    name
    myshopifyDomain
    email
  }
}
"""

res = requests.post(
    f"https://{SHOP_NAME}/admin/api/2024-04/graphql.json",
    headers=headers,
    json={"query": query}
)

print(res.status_code)
print(res.json())

200
{'data': {'shop': {'name': 'Local', 'myshopifyDomain': 'fh1z1f-5i.myshopify.com', 'email': 'info@seulementlocal.com'}}, 'extensions': {'cost': {'requestedQueryCost': 1, 'actualQueryCost': 1, 'throttleStatus': {'maximumAvailable': 2000.0, 'currentlyAvailable': 1999, 'restoreRate': 100.0}}}}


In [56]:
# ----------------------------------------
# 1. PRODUCTS_DF
# ----------------------------------------
QUERY_PRODUCTS = """
query getProducts($cursor: String) {
  products(first: 50, after: $cursor) {
    pageInfo {
      hasNextPage
    }
    edges {
      cursor
      node {
        id
        title
        vendor
        productType
        tags
        variants(first: 10) {
          edges {
            node {
              id
              sku
              price
            }
          }
        }
      }
    }
  }
}
"""

def fetch_products_df():
    cursor, has_next = None, True
    rows = []
    while has_next:
        r = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": QUERY_PRODUCTS, "variables": {"cursor": cursor}})
        r.raise_for_status()
        data = r.json()["data"]["products"]
        for edge in data["edges"]:
            cursor = edge["cursor"]
            node = edge["node"]
            for v_edge in node["variants"]["edges"]:
                variant = v_edge["node"]
                rows.append({
                    "product_id": node["id"],
                    "title": node["title"],
                    "vendor": node["vendor"],
                    "product_type": node["productType"],
                    "tags": ", ".join(node["tags"]),
                    "variant_id": variant["id"],
                    "sku": variant["sku"],
                    "price": float(variant["price"])
                })
        has_next = data["pageInfo"]["hasNextPage"]
    return pd.DataFrame(rows)

In [57]:
# ----------------------------------------
# 2. STOCK_DF
# ----------------------------------------
QUERY_STOCK = """
query getProductStock($cursor: String) {
  products(first: 50, after: $cursor) {
    pageInfo {
      hasNextPage
    }
    edges {
      cursor
      node {
        variants(first: 10) {
          edges {
            node {
              id
              sku
              inventoryItem {
                tracked
                inventoryLevels(first: 5) {
                  edges {
                    node {
                      location {
                        name
                      }
                      quantities(names: ["available"]) {
                        name
                        quantity
                      }
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}
"""

def fetch_stock_df():
    cursor, has_next = None, True
    rows = []
    while has_next:
        r = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": QUERY_STOCK, "variables": {"cursor": cursor}})
        r.raise_for_status()
        data = r.json()["data"]["products"]
        for edge in data["edges"]:
            cursor = edge["cursor"]
            for v_edge in edge["node"]["variants"]["edges"]:
                variant = v_edge["node"]
                inv = variant.get("inventoryItem")
                if inv and inv.get("tracked"):
                    for lvl in inv.get("inventoryLevels", {}).get("edges", []):
                        node = lvl["node"]
                        qty = next((q["quantity"] for q in node.get("quantities", []) if q["name"] == "available"), None)
                        rows.append({
                            "variant_id": variant["id"],
                            "sku": variant["sku"],
                            "location": node["location"]["name"],
                            "available": qty
                        })
        has_next = data["pageInfo"]["hasNextPage"]
    return pd.DataFrame(rows)

In [58]:
# ----------------------------------------
# 3. ORDERS_DF
# ----------------------------------------
QUERY_ORDERS = """
query getOrders($cursor: String) {
  orders(first: 50, after: $cursor, reverse: true) {
    pageInfo {
      hasNextPage
    }
    edges {
      cursor
      node {
        id
        name
        createdAt
        lineItems(first: 10) {
          edges {
            node {
              title
              quantity
              sku
              originalUnitPrice
              discountedUnitPrice
            }
          }
        }
      }
    }
  }
}
"""

def fetch_orders_df():
    cursor, has_next = None, True
    rows = []
    while has_next:
        r = requests.post(GRAPHQL_URL, headers=HEADERS, json={"query": QUERY_ORDERS, "variables": {"cursor": cursor}})
        r.raise_for_status()
        data = r.json()["data"]["orders"]
        for edge in data["edges"]:
            cursor = edge["cursor"]
            o = edge["node"]
            for li in o["lineItems"]["edges"]:
                item = li["node"]
                rows.append({
                    "order_id": o["id"],
                    "order_name": o["name"],
                    "created_at": o["createdAt"],
                    "title": item["title"],
                    "sku": item["sku"],
                    "quantity": item["quantity"],
                    "original_unit_price": float(item["originalUnitPrice"]),
                    "discounted_unit_price": float(item["discountedUnitPrice"])
                })
        has_next = data["pageInfo"]["hasNextPage"]
    return pd.DataFrame(rows)

In [75]:
# https://docs.google.com/spreadsheets/d/1NATO1x_KlDUQ6F0vbD32xqM_hPsS0NqDawo3_CdcO0o/edit?gid=1977368994#gid=1977368994
# ----------------------------------------
# 3. VENDORS_SPEC_DF -> GOOGLE SHEETS
# ----------------------------------------
def fetch_vendors_df():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("gspread_key.json", scope)
    client = gspread.authorize(creds)

    # Ouvre le Google Sheet avec l’ID du nouveau fichier
    sheet = client.open_by_key("1NATO1x_KlDUQ6F0vbD32xqM_hPsS0NqDawo3_CdcO0o")

    # Onglet cible à modifier si besoin
    worksheet = sheet.worksheet("Business_rules")

    # Transforme en DataFrame
    df = get_as_dataframe(worksheet, evaluate_formulas=True)
    df = df.dropna(how="all")  # Enlève les lignes vides
    df.columns = [c.strip() for c in df.columns]  # Nettoie les noms de colonnes
    return df

In [77]:
# ----------------------------------------
# 4. FETCH ALL TOGETHER
# ----------------------------------------
def fetch_datasets():
    products_df = fetch_products_df()
    stock_df = fetch_stock_df()
    orders_df = fetch_orders_df()
    vendors_df = fetch_vendors_df()
    return products_df, stock_df, orders_df, vendors_df

# Run it
products_df, stock_df, orders_df,vendors_df = fetch_datasets()

In [78]:
print("🔹 Products:", products_df.shape, "rows")
print("🔹 Stockm:", stock_df.shape, "rows")
print("🔹 Orders:", orders_df.shape, "rows")
print("🔹 Vendors:", vendors_df.shape, "rows")

🔹 Products: (790, 8) rows
🔹 Stockm: (790, 4) rows
🔹 Orders: (727, 8) rows
🔹 Vendors: (13, 8) rows


In [80]:
products_df.to_csv("products.csv", index=False)
stock_df.to_csv("stock.csv", index=False)
orders_df.to_csv("orders.csv", index=False)
vendors_df.to_csv("vendors.csv", index=False)